# データ解析

研究室の都合上、SASを使用してデータ分析を行った。

## ファイル読み込み

In [ ]:
OPTION NOSOURCE;
  
  %macro obscnt(data=,mvar=);
    %global &mvar;
    data _null_;
      set &data nobs=cnt;
      call symput("&mvar",left(put(cnt,8.)));
      stop;
    run;
  %mend;

## 一般化推定方程式(gee)と一般化線形混合効果モデル(glmm)を用いた分析

In [ ]:
%GLOBAL counter;
%let counter=0;

%macro doexe(filepath);
  %let counter=%eval(&counter+1);
  title &counter;
  filename FL &filepath;
  
  data start_prj_multi&counter;
    infile FL eov=eov;
    eov = 0 ;
    input @;
    if _N_ ^= 1 and eov ^= 1 then do;
      input outcome patient_id x1 w1 x2 w2;
      output;
    end;;
  run;
  
  %obscnt(data=start_prj_multi&counter,mvar=obs);

  data prj_multi&counter;
    set start_prj_multi&counter;
    if _N_ <= &obs * 0.8 then _Imputation_ = 4 ; else _Imputation_ = 5;
    if _N_ <= &obs * 0.6 then _Imputation_ = 3 ;
    if _N_ <= &obs * 0.4 then _Imputation_ = 2 ;
    if _N_ <= &obs * 0.2 then _Imputation_ = 1 ;
  run;
  
  data trans&counter;
    set prj_multi&counter;
    p1 = input(x1,8.);
    p2 = input(x2,8.);
    drop x1 x2;
  run;
  
  proc sort data=trans&counter;
    by _Imputation_ patient_id;
  run;
  
  data PRJ_MULTI_GEE&counter;
    set trans&counter;
    by _Imputation_ patient_id;
    if first.patient_id then SEQ = 0;
    SEQ+1;
  run;
  
  ods select none;
  ods output GEEFitCriteria = fitgee&counter;
  ods output GEEEmpPEst = pestgee&counter;
  proc genmod data = PRJ_MULTI_GEE&counter descending;
    BY _Imputation_;
    CLASS outcome patient_id SEQ;
    MODEL outcome(event='1') = p1 p2 / dist = binomial link = logit;
    REPEATED SUBJECT=patient_id / WITHINSUBJECT=SEQ TYPE=CS;
  run;
  ods select all;

  data fitgee_processed&counter;
    set fitgee&counter;
    where Criterion = 'QIC';
    fileid = &counter;
  run;
   
  data pest_geeforint&counter;
    set pestgee&counter;
    keep _Imputation_ Parm Estimate Stderr;
  run;
 
  ods select none;
  ods output ParameterEstimates = mianalyzed_gee&counter;
  proc mianalyze parms=pest_geeforint&counter;
    modeleffects Intercept p1 p2;
  run;
  ods select all;
  
  data mianalyzed_gee&counter;
    set mianalyzed_gee&counter;
    odds_ratio = exp(estimate);
    lcl=exp(estimate-probit(0.975)*stderr);
    ucl=exp(estimate+probit(0.975)*stderr);
    fileid = &counter;
  run;
  
  ods select none;
  ods output FitStatistics=fit_glmm&counter;
  ods output ParameterEstimates=pest_glmm&counter;
  PROC glimmix data =trans&counter method=quad INFOCRIT=Q;
    BY _Imputation_;
    CLASS outcome patient_id;
    MODEL outcome(event='1') = p1 p2 / dist = binary oddsratio link=logit s covb;
    RANDOM intercept / subject = patient_id;
  RUN;
  ods select all;
  
  /*適合度*/
  data fit_glmm_processed&counter;
    set fit_glmm&counter;
    where substr(Descr,1,4) = 'AIC ';
    fileid = &counter;
  run;
   
  ods select none;
  ods output ParameterEstimates = mianalyzed_glmm&counter;
  proc mianalyze parms=pest_glmm&counter;
    modeleffects Intercept p1 p2;
  run;
  ods select all;

  data mianalyzed_glmm&counter;
    set mianalyzed_glmm&counter;
    odds_ratio = exp(estimate);
    lcl=exp(estimate-probit(0.975)*stderr);
    ucl=exp(estimate+probit(0.975)*stderr);
    fileid = &counter;
  run;

%mend;

## miceで作成したデータの統合・分析
Miceで補完したデータを統合分析を行うため、$\ast$を指定し、上の関数で分析を行った．

In [ ]:
/* 5ファイルを統合する */
%doexe('<folder>\22_Model7mar_imp_001__IMPDATA*.dat');
/* 以下続く */

## 結果の出力

In [ ]:
DATA results_gee;
  set mianalyzed_gee:;
run;

proc sort data=results_gee;
  by fileid;
run;

PROC EXPORT DATA=results_gee
            OUTFILE= "<folder>\gee.csv"
            DBMS=CSV REPLACE;
RUN;

DATA results_glmm;
  set mianalyzed_glmm:;
run;

proc sort data=results_glmm;
  by fileid;
run;

PROC EXPORT DATA=results_glmm
            OUTFILE= "<folder>\glmm.csv"
            DBMS=CSV REPLACE;
RUN;